In [ ]:
import requests
from pydantic import BaseModel
import json
import datetime


class BitcludeClientException(Exception):
    pass


class CannotParseResponse(BitcludeClientException):
    pass


# https://github.com/samuelcolvin/pydantic/issues/1303
class HashableBaseModel(BaseModel):
    def __hash__(self):
        return hash((type(self),) + tuple(self.__dict__.values()))

class AccountHistoryItem(HashableBaseModel):
    currency1: str
    currency2: str
    amount: float
    time_close: datetime.datetime
    price: float
    fee_taker: int
    fee_maker: int
    type: str
    action: str




class BitcludeClient:
    def __init__(
        self, bitclude_url_base: str, bitclude_client_id: str, bitclude_client_key: str
    ):
        self.bitclude_url_base = bitclude_url_base
        self.bitclude_client_id = bitclude_client_id
        self.bitclude_client_key = bitclude_client_key

    def fetch_account_history(self):
        response = requests.get(
            self.bitclude_url_base,
            params={
                "method": "account",
                "action": "history",
                "id": self.bitclude_client_id,
                "key": self.bitclude_client_key,
            },
        )
        try:
            response_json = response.json()
        except json.decoder.JSONDecodeError:
            print(response.text)
            raise CannotParseResponse()
        if response_json["success"] == True:
            return [AccountHistoryItem(**item) for item in response_json["history"]]
        else:
            raise BitcludeClientException(response_json)

BITCLUDE_ID="xxx"
BITCLUDE_KEY="xxx"

client = BitcludeClient(
    bitclude_url_base="https://api.bitclude.com/",
    bitclude_client_id=BITCLUDE_ID,
    bitclude_client_key=BITCLUDE_KEY,
)


In [ ]:
account_history = client.fetch_account_history()

In [ ]:
import pandas as pd

account_history_dict = [item.dict() for item in account_history]

df = pd.DataFrame(account_history_dict, columns=["currency1", "currency2", "amount", "time_close", "price", "fee_taker", "fee_maker", "type", "action"])
# calculate amount * price * fee_taker / 1000 and append as a column
df["fee_paid"] = df["amount"] * df["price"] * df["fee_taker"] / 10000
df[df["action"] == "close"].head(50)
# count daily sum of fee_paid

# convert time_close to datetime
df["time_close"] = pd.to_datetime(df["time_close"])

# show last 7 days in timezone Europe/Warsaw

df[df["time_close"] > datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=7)].groupby(["time_close"]).sum()["fee_paid"].plot()

In [ ]:
30 / 10000